# 处理多个序列

In [27]:
from pprint import pprint as print
from transformers import AutoModelForSequenceClassification, AutoTokenizer
checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [38]:
s = 'Today is a nice day!'
inputs = tokenizer(s, return_tensors='pt')
print(inputs)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[ 101, 2651, 2003, 1037, 3835, 2154,  999,  102]])}


可以看到，这里的inputs包含了两个部分：`input_ids`和`attention_mask`.

模型可以直接接受`input_ids`并输出：

In [39]:
model(inputs.input_ids).logits

tensor([[-4.3232,  4.6906]], grad_fn=<AddmmBackward>)

也可以通过`**inputs`同时接受`inputs`所有的属性：

In [40]:
model(**inputs).logits

tensor([[-4.3232,  4.6906]], grad_fn=<AddmmBackward>)

上面两种方式的结果是一样的。

## 但是当我们需要同时处理**多个序列**时，情况就有变了！

In [42]:
ss = ['Today is a nice day!',
      'But what about tomorrow? Im not sure.']
inputs = tokenizer(ss, padding=True, return_tensors='pt')
print(inputs)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[  101,  2651,  2003,  1037,  3835,  2154,   999,   102,     0,     0,
             0],
        [  101,  2021,  2054,  2055,  4826,  1029, 10047,  2025,  2469,  1012,
           102]])}


In [43]:
model(inputs.input_ids).logits  # 原本是 [-4.3232,  4.6906]

tensor([[-4.1957,  4.5675],
        [ 3.9803, -3.2120]], grad_fn=<AddmmBackward>)

发现，第一个句子的`logits`变了！

这是因为在padding之后，第一个句子的encoding变了，多了很多0， 而self-attention会attend到所有的index的值，因此结果就变了。

这时，就需要我们不仅仅是传入`input_ids`，还需要给出`attention_mask`，这样模型就会在attention的时候，不去attend被mask掉的部分。

通过`**inputs`，我们实际上就把`attention_mask`也传进去了:

In [44]:
model(**inputs).logits

tensor([[-4.3232,  4.6906],
        [ 3.9803, -3.2120]], grad_fn=<AddmmBackward>)

model直接call接受的参数见：\
https://huggingface.co/transformers/master/model_doc/bert.html#bertmodel